In [11]:
import onnxruntime as rt
import torch
from coco_eval import CocoEvaluator
from tqdm.notebook import tqdm

import numpy as np
import onnx
from onnxruntime import quantization

In [12]:
yolon_path = "./trained_yolov10n/weights/best.onnx"
nanodet_path = "nanodet.onnx"

In [13]:
from roboflow import Roboflow
rf = Roboflow(api_key="xrjb7ahPJgT610pkOEf2")
project = rf.workspace("mooncrater").project("mooncrater")
dataset = project.version(3).download("coco")

loading Roboflow workspace...
loading Roboflow project...


In [14]:
import os
import torchvision

# settings
ANNOTATION_FILE_NAME = "_annotations.coco.json"
TRAIN_DIRECTORY = os.path.join(dataset.location, "train")
VAL_DIRECTORY = os.path.join(dataset.location, "valid")
TEST_DIRECTORY = os.path.join(dataset.location, "test")

class CocoDetection(torchvision.datasets.CocoDetection):
    def __init__(
        self,
        image_directory_path: str,
        # onnx_model_path: str,
        train: bool = True
    ):
        annotation_file_path = os.path.join(image_directory_path, ANNOTATION_FILE_NAME)
        super(CocoDetection, self).__init__(image_directory_path, annotation_file_path)

    def __getitem__(self, idx):
        images, annotations = super(CocoDetection, self).__getitem__(idx)
        image_id = self.ids[idx]
        annotations = {'image_id': image_id, 'annotations': annotations}
        
        return images, annotations


TRAIN_DATASET = CocoDetection(
    image_directory_path=TRAIN_DIRECTORY,
    train=True)
VAL_DATASET = CocoDetection(
    image_directory_path=VAL_DIRECTORY,
    train=False)
TEST_DATASET = CocoDetection(
    image_directory_path=TEST_DIRECTORY,
    train=False)

print("Number of training examples:", len(TRAIN_DATASET))
print("Number of validation examples:", len(VAL_DATASET))
print("Number of test examples:", len(TEST_DATASET))

loading annotations into memory...
Done (t=0.03s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Number of training examples: 1479
Number of validation examples: 80
Number of test examples: 20


In [23]:
class QuantizationDataReader(quantization.CalibrationDataReader):
    def __init__(self, test_ds, batch_size, input_name):

        self.test_ds = test_ds

        self.input_name = input_name
        self.datasize = len(self.test_ds)

        self.enum_data = iter(self.test_ds)

    def to_numpy(self, image):
        image = np.array(image)
        image = image.astype(np.float32) / 255.0
        # print(image)
        image = np.transpose(image, (2, 0, 1))  # Change data layout from HWC to CHW
        image = np.expand_dims(image, axis=0) 

        return image

    def get_next(self):
        batch = next(self.enum_data, None)
        if batch is not None:
          return {self.input_name: self.to_numpy(batch[0])}
        else:
          return None

    def rewind(self):
        self.enum_data = iter(self.torch_dl)



In [16]:
TEST_DATASET[2]

(<PIL.Image.Image image mode=RGB size=640x640>,
 {'image_id': 2,
  'annotations': [{'id': 7,
    'image_id': 2,
    'category_id': 3,
    'bbox': [74, 238, 35, 43.5],
    'area': 1522.5,
    'segmentation': [],
    'iscrowd': 0},
   {'id': 8,
    'image_id': 2,
    'category_id': 3,
    'bbox': [489, 261, 33.5, 24.5],
    'area': 820.75,
    'segmentation': [],
    'iscrowd': 0},
   {'id': 9,
    'image_id': 2,
    'category_id': 3,
    'bbox': [3, 304, 49, 61.5],
    'area': 3013.5,
    'segmentation': [],
    'iscrowd': 0},
   {'id': 10,
    'image_id': 2,
    'category_id': 3,
    'bbox': [397, 324, 68.5, 50],
    'area': 3425,
    'segmentation': [],
    'iscrowd': 0},
   {'id': 11,
    'image_id': 2,
    'category_id': 3,
    'bbox': [114, 344, 48.5, 30.5],
    'area': 1479.25,
    'segmentation': [],
    'iscrowd': 0},
   {'id': 12,
    'image_id': 2,
    'category_id': 3,
    'bbox': [43, 346, 56.5, 52],
    'area': 2938,
    'segmentation': [],
    'iscrowd': 0},
   {'id': 13,


In [24]:

# onnx_model = rt.InferenceSession(nanodet_path)


In [25]:
from PIL import ImageDraw

In [32]:
import time
def inf_on_image(input_pil_image, onnx_model):
    
    image = np.array(input_pil_image)

    # Preprocess the images as required by your ONNX model
    # This might involve resizing, normalization, adding a batch dimension, etc.
    # The exact preprocessing steps will depend on your specific model

    # Convert the images to a format that can be used as input to the ONNX model
    # print(image)
    image = image.astype(np.float32) / 255.0
    # print(image)
    image = np.transpose(image, (2, 0, 1))  # Change data layout from HWC to CHW
    image = np.expand_dims(image, axis=0)  # Add batch dimension

    # Run the ONNX model
    input_name = onnx_model.get_inputs()[0].name

    start = time.perf_counter()
    predictions = onnx_model.run(None, {input_name: image})
    end = (time.perf_counter() - start) * 1000

    return predictions, end

def draw_preds(input_pil_image, predictions, conf=0.70):
    draw = ImageDraw.Draw(input_pil_image)

    # Define a color for each label
    colors = {0: "green", 1: "blue", 2: "red"}

    for prediction in predictions[0][0]:

        x1, y1, x2, y2, score, label = prediction
        
        #print(score, label)

        if score > conf:
            # Draw the bounding box on the image
            x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])
            draw.rectangle([x1, y1, x2, y2], outline=colors[int(label)], width=2)

    # Display the image
    input_pil_image.show()
    

In [33]:
test_img, _ = TEST_DATASET[4]

preds = inf_on_image(test_img)

print(preds[0].shape)

draw_preds(test_img, preds, 0.25)

TypeError: inf_on_image() missing 1 required positional argument: 'onnx_model'

In [34]:
TEST_DATASET.coco.dataset['annotations']

[{'id': 0,
  'image_id': 0,
  'category_id': 2,
  'bbox': [100, 402, 110.5, 38.5],
  'area': 4254.25,
  'segmentation': [],
  'iscrowd': 0},
 {'id': 1,
  'image_id': 1,
  'category_id': 2,
  'bbox': [41, 303, 100.5, 29.5],
  'area': 2964.75,
  'segmentation': [],
  'iscrowd': 0},
 {'id': 2,
  'image_id': 1,
  'category_id': 2,
  'bbox': [510, 330, 123.5, 46],
  'area': 5681,
  'segmentation': [],
  'iscrowd': 0},
 {'id': 3,
  'image_id': 1,
  'category_id': 2,
  'bbox': [342, 339, 65, 33.5],
  'area': 2177.5,
  'segmentation': [],
  'iscrowd': 0},
 {'id': 4,
  'image_id': 1,
  'category_id': 2,
  'bbox': [55, 361, 91.5, 34.5],
  'area': 3156.75,
  'segmentation': [],
  'iscrowd': 0},
 {'id': 5,
  'image_id': 1,
  'category_id': 2,
  'bbox': [291, 396, 262.5, 79],
  'area': 20737.5,
  'segmentation': [],
  'iscrowd': 0},
 {'id': 6,
  'image_id': 1,
  'category_id': 2,
  'bbox': [4, 427, 218.5, 50.5],
  'area': 11034.25,
  'segmentation': [],
  'iscrowd': 0},
 {'id': 7,
  'image_id': 2,


In [42]:
import time

def calculate_metrics(inference_func, session):
    evaluator = CocoEvaluator(coco_gt=TEST_DATASET.coco, iou_types=["bbox"])

    print("Running evaluation...")

    total_model_time = 0.0

    for idx, (input_pil_image, annotation) in enumerate(TEST_DATASET):
        
        image_id = annotation['image_id']

        predictions, elapsed_time = inference_func(input_pil_image, session)
        total_model_time += elapsed_time
        coco_results = []
        for prediction in predictions[0][0]:

            # Extract the bounding box coordinates, confidence score, and class label
            x1, y1, x2, y2, score, label = prediction
            
            # Convert the bounding box from (x1, y1, x2, y2) to (x, y, width, height)
            box = [x1, y1, x2 - x1, y2 - y1]

            print(score,label)

            if score > 0.25:
                # Append the result
                coco_results.append({
                    "image_id": image_id,
                    "category_id": int(label)+1,
                    "bbox": box,
                    "score": float(score),
                })
        
        if len(coco_results) > 0:
            evaluator.update(coco_results)
    


    print(f"Elapsed time: {total_model_time} seconds")
    

    evaluator.synchronize_between_processes()
    evaluator.accumulate()
    evaluator.summarize()

In [36]:
calculate_metrics(inf_on_image)


Running evaluation...


NameError: name 'onnx_model' is not defined

In [58]:
test_img, _ = TEST_DATASET[2]

In [17]:
sess_options = rt.SessionOptions()
sess_options.graph_optimization_level = rt.GraphOptimizationLevel.ORT_ENABLE_ALL

# Specify DNNL as the execution provider
# session = rt.InferenceSession(yolon_path, sess_options, providers=['DnnlExecutionProvider'])
session = rt.InferenceSession(yolon_path, sess_options, providers=['CPUExecutionProvider'])

In [60]:
onnx_model = rt.InferenceSession(yolon_path)


In [61]:
onnx_model.get_inputs()[0].name

'images'

In [30]:
def benchmark_model_time(session, num=1000):
    total_time = 0.0
    num_runs = 30
    for i in range(0,num_runs):
        _, elp_time = inf_on_image(test_img, session)
        total_time += elp_time
    avg_time_ms = total_time / num_runs
    return avg_time_ms

In [68]:
avg_time_ms

61.02335736650275

In [54]:
qdr = QuantizationDataReader(TEST_DATASET, batch_size=1, input_name=session.get_inputs()[0].name)

In [46]:
qdr.get_next()

{'images': array([[[[0.01176471, 0.01176471, 0.01176471, ..., 0.02745098,
           0.02745098, 0.02745098],
          [0.01176471, 0.01176471, 0.01176471, ..., 0.02745098,
           0.02745098, 0.02745098],
          [0.01176471, 0.01176471, 0.01176471, ..., 0.02745098,
           0.02745098, 0.02745098],
          ...,
          [0.2627451 , 0.23921569, 0.21176471, ..., 0.12156863,
           0.10588235, 0.08627451],
          [0.24313726, 0.23137255, 0.21568628, ..., 0.12156863,
           0.10196079, 0.08627451],
          [0.21176471, 0.21176471, 0.21176471, ..., 0.12156863,
           0.10196079, 0.08627451]],
 
         [[0.03137255, 0.03137255, 0.03137255, ..., 0.04705882,
           0.04705882, 0.04705882],
          [0.03137255, 0.03137255, 0.03137255, ..., 0.04705882,
           0.04705882, 0.04705882],
          [0.03137255, 0.03137255, 0.03137255, ..., 0.04705882,
           0.04705882, 0.04705882],
          ...,
          [0.3137255 , 0.2901961 , 0.2627451 , ..., 0.133

In [48]:
yolon_path_prep = "yolon_prep.onnx"
quantization.shape_inference.quant_pre_process(yolon_path, yolon_path_prep, skip_symbolic_shape=False)

In [55]:
q_static_opts = {"ActivationSymmetric":False,
                "WeightSymmetric":True}
if torch.cuda.is_available():
    q_static_opts = {"ActivationSymmetric":True,
                    "WeightSymmetric":True}

model_int8_path = 'yolon_int8.onnx'
quantized_model = quantization.quantize_static(model_input=yolon_path,
                                               model_output=model_int8_path,
                                               calibration_data_reader=qdr,
                                               extra_options=q_static_opts)

In [56]:
int8_model = rt.InferenceSession(model_int8_path)

In [51]:
benchmark_model_time(int8_model, num=100)

134.7144072663165

In [59]:
inf_on_image(test_img,int8_model)

([array([[[  0.      ,   0.      ,  16.95529 ,  11.303527,   0.      ,
             0.      ],
          [  0.      ,   0.      ,  16.95529 ,  11.303527,   0.      ,
             0.      ],
          [  0.      ,   0.      ,  16.95529 ,  11.303527,   0.      ,
             0.      ],
          ...,
          [113.03527 ,   0.      , 186.5082  ,  16.95529 ,   0.      ,
             0.      ],
          [113.03527 ,   0.      , 186.5082  ,  16.95529 ,   0.      ,
             0.      ],
          [113.03527 ,   0.      , 186.5082  ,  16.95529 ,   0.      ,
             0.      ]]], dtype=float32)],
 239.04588799996418)